## Question 1

Given an integer array nums, reorder it such that nums[0] < nums[1] > nums[2] < nums[3]....

You may assume the input array always has a valid answer.

In [ ]:
class Solution:
    def wiggleSort(self, nums: List[int]) -> None:
        """
        Do not return anything, modify nums in-place instead.
        """
        n = len(nums)
        nums.sort()
        mid = (n - 1) // 2
        nums[::2], nums[1::2] = nums[mid::-1], nums[:mid:-1]

## Question 2

In LeetCode Store, there are n items to sell. Each item has a price. However, there are some special offers, and a special offer consists of one or more different kinds of items with a sale price.

You are given an integer array price where price[i] is the price of the ith item, and an integer array needs where needs[i] is the number of pieces of the ith item you want to buy.

You are also given an array special where special[i] is of size n + 1 where special[i][j] is the number of pieces of the jth item in the ith offer and special[i][n] (i.e., the last integer in the array) is the price of the ith offer.

Return the lowest price you have to pay for exactly certain items as given, where you could make optimal use of the special offers. You are not allowed to buy more items than you want, even if that would lower the overall price. You could use any of the special offers as many times as you want.

In [ ]:
class Solution:
    def shoppingOffers(self, price: List[int], special: List[List[int]], needs: List[int]) -> int:
        n = len(price)
        @lru_cache(maxsize=None)
        def dfs(needs):
            ans = sum([i*j for i, j in zip(price, needs)]) 
            cur = sys.maxsize
            for s in special:
                new_needs, ok = [], True
                for i in range(n):
                    need, give = needs[i], s[i]
                    if need < give:  # if over purchase, ignore this combination
                        ok = False
                        break
                    new_needs.append(need-give)    
                if ok: cur = min(cur, dfs(tuple(new_needs)) + s[-1])
            return min(ans, cur)
        return dfs(tuple(needs))

## Question 3

You are given two 0-indexed strings source and target, both of length n and consisting of lowercase English letters. You are also given two 0-indexed character arrays original and changed, and an integer array cost, where cost[i] represents the cost of changing the character original[i] to the character changed[i].

You start with the string source. In one operation, you can pick a character x from the string and change it to the character y at a cost of z if there exists any index j such that cost[j] == z, original[j] == x, and changed[j] == y.

Return the minimum cost to convert the string source to the string target using any number of operations. If it is impossible to convert source to target, return -1.

Note that there may exist indices i, j such that original[j] == original[i] and changed[j] == changed[i].

In [ ]:
from queue import PriorityQueue

class Solution:
    def minimumCost(self, source: str, target: str, original: List[str], changed: List[str], cost: List[int]) -> int:
        graph = {}
        for i, elem in enumerate(original):
            if elem not in graph:
                graph[elem] = {}
            graph[elem][changed[i]] = min(cost[i], graph[elem].get(changed[i], float("inf")))

        total = 0
        memo = {} # memoizes the shortest path from source to target
        
        for i, letter in enumerate(source):
            if letter != target[i]:
                result = self.find_path(source[i], target[i], graph, memo)
                if result == -1:
                    return -1
                total += result
        return total
    
    
    def find_path(self, source, target, graph, memo):
        if (source, target) in memo:
            return memo[(source, target)]
        
        pq = PriorityQueue()
        pq.put((0, source))
        seen = set()
        
        while not pq.empty():
            cost, letter = pq.get()
            if letter == target:
                memo[(source, target)] = cost
                return cost
            if letter not in seen:
                seen.add(letter)
                memo[(source, letter)] = cost
                edges = graph.get(letter, [])
                for edge in edges:
                    pq.put((cost + edges[edge], edge))
        return -1